# Pre-processing of data
## Sources
https://storage.googleapis.com/earthenginepartners-hansen/GFC-2020-v1.8/Hansen_GFC-2020-v1.8_lossyear_50N_080W.tif

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from osgeo import gdal
import os

from rasterstats import zonal_stats

from rasterio import open as r_open
from osgeo import gdal
import rasterio
from affine import Affine
from subprocess import Popen

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

os.getcwd()

'C:\\Users\\RoseDeterman\\2021_Fall\\EE508_data\\Final_2'

In [2]:
import winsound
duration = 250
freq = 350
winsound.Beep(freq, duration)

# Parcels

In [3]:
#import
parcels = gpd.read_file("data/L3_SHP_M101_FRANKLIN\L3_SHP_M101_Franklin/M101TaxPar_CY21_FY22.shp")

#drop rows with an unknown or no parcel id
lgc = (parcels["MAP_PAR_ID"] == "UNKNOWN") | (parcels["MAP_PAR_ID"].isnull())
parcels = parcels[~lgc]

## Join with assessor's data

In [4]:
#import 
asr = pd.read_csv("data/assessor/CDM_GIS_output_edit.csv")

#rename id column
asr.rename(columns = {'Parcel ID Number':'MAP_PAR_ID'}, inplace = True)
#trim white space from id
asr["MAP_PAR_ID"] = asr["MAP_PAR_ID"].str.strip()

#join parcels with the assesed value data
parcels = parcels.set_index('MAP_PAR_ID').join(asr.set_index('MAP_PAR_ID'))

#keep only columns related to assessed value
parcels = parcels[[ "Total Assessed Value","Primary Land Use Code (LUC)", "geometry"]]
parcels.columns = [ 't_as_val','LUC', 'geometry']

## Calculate cost per area

In [5]:
#Calculate Parcel area
#convert to equal area projection and calculate area
parcels['area'] = parcels.to_crs("EPSG:2163").geometry.area

#Calculate cost per area
parcels['val_per_ar'] = parcels['t_as_val']/parcels['area']

# Open Space


In [6]:
parcels.reset_index(inplace=True)

In [7]:
open_poly = gpd.read_file("data/openspace/OPENSPACE_POLY.shp")
open_poly = open_poly[open_poly["TOWN_ID"] == 101]
open_poly = open_poly[["POLY_ID", "SITE_NAME", "FEE_OWNER", "PRIM_PURP", "PUB_ACCESS","LEV_PROT", "geometry"]]

winsound.Beep(freq, duration)

In [8]:
inters = gpd.overlay(parcels,open_poly, how='intersection', make_valid=True, keep_geom_type = True)

In [9]:
parcels = parcels.merge(inters,
               left_on='MAP_PAR_ID', right_on='MAP_PAR_ID', how='left')
parcels = parcels.set_geometry("geometry_x")

# Forest Change

In [10]:
#Clip to extent of town borders

ds = gdal.Open('data/forest_change/Hansen_GFC-2020-v1.8_gain_50N_080W.tif')
ds = gdal.Translate('data/forest_change/gain_clip.tif', ds, projWin = [-71.45590532518631,
                                                                       42.1395760939559,
                                                                       -71.3591633138789,
                                                                       42.034819575711865])
ds = None

ds = gdal.Open('data/forest_change/Hansen_GFC-2020-v1.8_lossyear_50N_080W.tif')
ds = gdal.Translate('data/forest_change/loss_clip.tif', ds, projWin = [-71.45590532518631,42.1395760939559,
                                                                     -71.3591633138789,42.034819575711865])
ds = None 

In [11]:
step1 = gdal.Open('data/forest_change/loss_clip.tif', gdal.GA_ReadOnly)

GT_input = step1.GetGeoTransform()
afn = Affine.from_gdal(*GT_input)

step2 = step1.GetRasterBand(1)
img_as_array = step2.ReadAsArray()
size1,size2=img_as_array.shape

output = np.where(img_as_array != 0, 1, 0)


dst_crs='EPSG:4326'

output = np.float32(output)
with rasterio.open(
    'data/forest_change/loss_clip_recode.tif',
   'w',
    driver='GTiff',
    height=output.shape[0],
    width=output.shape[1],
    count=1,
    dtype=np.float32,
    crs=dst_crs,
    transform=afn,
) as dest_file:
    dest_file.write(output, 1)
dest_file.close()


In [12]:
parcels = parcels.to_crs("EPSG:4326")
x= zonal_stats(parcels,
            "data/forest_change/loss_clip_recode.tif",
            stats="mean", all_touched=True)
x = pd.DataFrame(x, index=parcels.index)
x.columns = x.columns.map(lambda x: str(x) + '_loss')
parcels = parcels.join(x)

x= zonal_stats(parcels,
            "data/forest_change/gain_clip.tif",
            stats="mean", all_touched=True)
x = pd.DataFrame(x, index=parcels.index)
x.columns = x.columns.map(lambda x: str(x) + '_gain')
parcels = parcels.join(pd.DataFrame(x, index=parcels.index))

winsound.Beep(freq, duration)

OverflowError: cannot convert float infinity to integer

# Priority Habitat
https://lpsmlgeo.github.io/2019-09-22-binary_mask/

In [ ]:
raster_path = "data/forest_change/gain_clip.tif"
with rasterio.open(raster_path, "r") as src:
    raster_img = src.read()
    raster_meta = src.meta

In [ ]:
shape_path = "data/prihab/PRIHAB_POLY.shp"
train_df = gpd.read_file(shape_path)

In [ ]:
train_df= train_df.to_crs("EPSG:4326")

In [ ]:
print("CRS Raster: {}, CRS Vector {}".format(train_df.crs, src.crs))

In [ ]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # Convert polygons to the image CRS
        poly_pts.append(~transform * tuple(i))
        
    # Generate a polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

# Generate Binary maks

poly_shp = []
im_size = (src.meta['height'], src.meta['width'])
for num, row in train_df.iterrows():
    if row['geometry'].geom_type == 'Polygon':
        poly = poly_from_utm(row['geometry'], src.meta['transform'])
        poly_shp.append(poly)
    else:
        for p in row['geometry']:
            poly = poly_from_utm(p, src.meta['transform'])
            poly_shp.append(poly)

mask = rasterize(shapes=poly_shp,
                 out_shape=im_size)

winsound.Beep(freq, duration)

In [ ]:
#mask = mask.astype("uint8")
save_path = "data/prihab/prihab.tif"
bin_mask_meta = src.meta.copy()
bin_mask_meta.update({'count': 1})
with rasterio.open(save_path, 'w', **bin_mask_meta) as dst:
    dst.write(mask, 1)

In [ ]:
x= zonal_stats(parcels,
            "data/prihab/prihab.tif",
            stats="mean", all_touched=True)
x = pd.DataFrame(x, index=parcels.index)
x.columns = x.columns.map(lambda x: str(x) + '_prihab')
parcels = parcels.join(x)

winsound.Beep(freq, duration)

In [ ]:
parcels

# Land Cover

## Define function that recodes rasters to 0/1 based on landcover type of interest

**Developed**  
2 Impervious
5 Developed open space  

**Wetland**  
13 	Palustrine Forested Wetland (C-CAP)  
14 	Palustrine Scrub/Shrub Wetland (C-CAP)  
15 	Palustrine Emergent Wetland (C-CAP)  
16 	Estuarine Forested Wetland (C-CAP)  
17 	Estuarine Scrub/Shrub Wetland (C-CAP)  
18 	Estuarine Emergent Wetland (C-CAP)  

**Forest**  
9 	Deciduous Forest  
10 	Evergreen Forest  

In [ ]:
# recode based on cover code
def developed(img_as_array):
    condition = np.where(((img_as_array == 2) | (img_as_array ==5)), 1, 0)
    return condition
def wetland(img_as_array):
    condition = np.where(((img_as_array >= 12) & (img_as_array <=18)), 1, 0)
    return condition
def forest_shrub(img_as_array):
    condition = np.where(((img_as_array == 9) | (img_as_array ==10)), 1, 0)
    return condition

#do the recoding of the raster
def recode_lc(to_filepath, lc):
    step1 = gdal.Open('data/2016_HighResLandCover/Job680452_2016_HighResLandCover.tif', gdal.GA_ReadOnly)

    GT_input = step1.GetGeoTransform()
    afn = Affine.from_gdal(*GT_input)

    print("step1 done", sep = " ")
    step2 = step1.GetRasterBand(1)
    img_as_array = step2.ReadAsArray()
    size1,size2=img_as_array.shape
    print("step2 done", sep = " ")
    
    if lc == "developed":
        output = developed(img_as_array)
    elif lc == "wetland":
        output = wetland(img_as_array)
    elif lc == "forest_shrub":
        output = forest_shrub(img_as_array)
    else:
        print("unknown recode")

    print("recode done")
    
    dst_crs='EPSG:3586'
    output = np.float32(output)
    with rasterio.open(
        to_filepath,
        'w',
        driver='GTiff',
        height=output.shape[0],
        width=output.shape[1],
        count=1,
        dtype=np.float32,
        crs=dst_crs,
        transform=afn,
    ) as dest_file:
        dest_file.write(output, 1)
    dest_file.close()

## Do the recoding

In [ ]:
to_filepath = 'data/2016_HighResLandCover/developed.tif'
recode_lc(to_filepath, "developed")

to_filepath = 'data/2016_HighResLandCover/wetland.tif'
recode_lc(to_filepath, "wetland")

to_filepath = 'data/2016_HighResLandCover/forest_shrub.tif'
recode_lc(to_filepath, "forest_shrub")

In [ ]:
#reproject shapes to raster projection
parcels = parcels.to_crs("EPSG:3586")

#CALCULATE FREST/SHRUB COVER
x= zonal_stats(parcels,
            'data/2016_HighResLandCover/forest_shrub.tif',
            stats="mean", all_touched=True)
x = pd.DataFrame(x, index=parcels.index)
x.columns = x.columns.map(lambda x: str(x) + '_for')
parcels = parcels.join(x)
print("forest done")

winsound.Beep(freq, duration)

In [ ]:
x= zonal_stats(parcels,
            'data/2016_HighResLandCover/wetland.tif',
            stats="mean", all_touched=True)
x = pd.DataFrame(x, index=parcels.index)
x.columns = x.columns.map(lambda x: str(x) + '_wet')
parcels = parcels.join(x)
print("wetland done")

x= zonal_stats(parcels,
            'data/2016_HighResLandCover/developed.tif',
            stats="mean", all_touched=True)
x = pd.DataFrame(x, index=parcels.index)
x.columns = x.columns.map(lambda x: str(x) + '_dev')
parcels = parcels.join(x)
print("developed done")

winsound.Beep(freq, duration)

# Join CH61

In [ ]:
ch61 = pd.read_csv("data/assessor/CH61.csv")

ch61 = ch61[["PARCEL_NUM", "CH61_TYPE"]]

ch61.columns = ['MAP_PAR_ID','CH61_TYPE']
ch61["rep"] = "-000"
ch61['MAP_PAR_ID'] = ch61['MAP_PAR_ID'].str.cat([ch61['rep']])

In [ ]:
ch61['MAP_PAR_ID'].head()

In [ ]:
parcels["MAP_PAR_ID"].head()

In [ ]:
parcels = parcels.set_index("MAP_PAR_ID").join(ch61.set_index("MAP_PAR_ID"), how = "left")

# Export All

In [ ]:
#parcels = parcels[["MAP_PAR_ID","t_as_val_x","area_x","val_per_ar_x","PRIHAB_ID","mean_fors","mean_wetl","mean_dev","geometry_x", "mean", "mean_gain"]]

In [ ]:
parcels.reset_index(inplace = True)

In [ ]:
parcels[['MAP_PAR_ID', 't_as_val_x', 'LUC_x', "CH61_TYPE", 'geometry_x', 'area_x','POLY_ID', 'SITE_NAME',
       'FEE_OWNER', 'PRIM_PURP', 'PUB_ACCESS', 'LEV_PROT',
       'mean_loss', 'mean_gain', 'mean_prihab', 'mean_for', 'mean_wet',
       'mean_dev']].to_file("data/parcels.shp")